# Import Libraries

In [ ]:
import numpy as np
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Input, Lambda, Flatten, Dense
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential